In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [73.4 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [920 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,017 kB]
Hit:13 http://ppa.launchpad.net/graphics

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-03-20 08:35:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.95MB/s    in 0.2s    

2023-03-20 08:35:37 (5.95 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Office_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.|2015-08-31 00:00:00|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| 

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R18RVCKGH1SSI9|          5|            0|          0|   N|                Y|
|R3L4L6LW1PUOFY|          5|            0|          1|   N|                Y|
|R2J8AWXWTDX2TF|          5|            0|          0|   N|                Y|
|R1PR37BR7G3M6A|          1|            2|          3|   N|                Y|
|R3BDDDZMZBZDPU|          4|            0|          0|   N|                Y|
| R8T6MO75ND212|          5|            0|          0|   N|                Y|
|R2YWMQT2V11XYZ|          5|            0|          0|   N|                N|
|R1V2HYL6OI9V39|          5|            6|          6|   N|                Y|
|R3BLQBKUNXGFS4|          5|            0|          0|   N|                Y|
|R17MOWJCAR9Y8Q|          5|            0|          0|   N|     

In [7]:
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [8]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
vine_df = vine_df.withColumn("star_rating",col("star_rating").cast(IntegerType()))
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [9]:
vine_votes_df = vine_df.filter("total_votes>=20")
vine_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|
| RLV4XQ49EBYYU|          1|           56|         65|   N|                Y|
| RPBPAJKFD0T26|          5|           63|         63|   N|                Y|
|R1IHO8C56KBN6C|          1|           61|         64|   N|     

In [10]:
vine_helpful_df = vine_votes_df.filter("helpful_votes/total_votes>=0.5")
vine_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|
| RLV4XQ49EBYYU|          1|           56|         65|   N|                Y|
| RPBPAJKFD0T26|          5|           63|         63|   N|                Y|
|R1IHO8C56KBN6C|          1|           61|         64|   N|     

In [11]:
vine_program_df = vine_helpful_df.filter("vine == 'Y'")
vine_program_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3JIGR122X50ZV|          5|         2280|       2314|   Y|                N|
|R3KV8P8WLD2KKC|          5|          263|        286|   Y|                N|
|R2LFY2ZPJ3J7RH|          4|           36|         38|   Y|                N|
|R3P01OXMA0U0CA|          5|          114|        117|   Y|                N|
|R2SCMXBD9OQP8Z|          2|           22|         26|   Y|                N|
|R2FGS9ECOK5IQN|          5|           22|         23|   Y|                N|
| R1V9CO41XS6QY|          2|           56|         58|   Y|                N|
|R1LEP1JGXRY2YT|          2|           18|         20|   Y|                N|
|R1ZWQJ3ZBSIZ1K|          4|           19|         21|   Y|                Y|
| RX1G4FUE5HZGN|          3|           52|         58|   Y|     

In [22]:
# total number of reviews, number of 5-star reviews, percentage of 5-star reviews
vine_program_df.count()

969

In [23]:
five_paid = vine_program_df.filter(vine_program_df.star_rating == 5).count()
five_paid

430

In [12]:
not_vine_program_df = vine_helpful_df.filter("vine == 'N'")
not_vine_program_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|
| RLV4XQ49EBYYU|          1|           56|         65|   N|                Y|
| RPBPAJKFD0T26|          5|           63|         63|   N|                Y|
|R1IHO8C56KBN6C|          1|           61|         64|   N|     

In [14]:
# total number of reviews, number of 5-star reviews, percentage of 5-star reviews
not_vine_program_df.count()

43745

In [24]:
five_unpaid = not_vine_program_df.filter(not_vine_program_df.star_rating == 5).count()
five_unpaid

19233